In [ ]:
import pandas as pd
import os, sys
from egfet.load_data import load_egfet_dataset
# Load the feature importances CSV file
feature_importances_df = pd.read_csv('/home/jalex/Projects/aros/histones/Histones/paper_results_6/2KPEG_Hapt_Train_Mag_Control/run_0/cv_results/top_features.csv')
# Sort the features by importance score in descending order
sorted_features = feature_importances_df.sort_values(by='count', ascending=False)
# Get the top features by importance score
top_features = sorted_features.head(10)  # Change the number 10 to the desired number of top features

# Print the top features
print(top_features)
features_df = pd.read_csv('~/Data/aros/Histones/PavlidisGroup/EGFET Processed Data/downsample_1_magnitude_controlled_exclude_concentrations_0nM_features3.csv')
data = features_df.drop(columns=["Unnamed: 0"])
# first 4 columns are Experiment, Concentration, Trial, Cycle -- rename them
data.columns = ["Experiment", "Concentration", "Trial", "Cycle"] + list(
    data.columns[4:]
)
data = data.set_index(["Experiment", "Concentration", "Trial", "Cycle"])

   Unnamed: 0                                            feature  \
0           0  Drain Current (nA)__energy_ratio_by_chunks__nu...   
1           1  Drain Current (nA)__agg_linear_trend__attr_"sl...   
2           2  Drain Current (nA)__agg_autocorrelation__f_agg...   
3           3  Drain Current (nA)__agg_linear_trend__attr_"sl...   
4           4  Drain Current (nA)__cwt_coefficients__coeff_0_...   
5           5  Drain Current (nA)__agg_autocorrelation__f_agg...   
6           6  Drain Current (nA)__fft_coefficient__attr_"rea...   
7           7                                           0_LPCC_0   
8           8  Drain Current (nA)__cwt_coefficients__coeff_1_...   
9           9  Drain Current (nA)__cwt_coefficients__coeff_6_...   

   feature_importance  
0            0.096501  
1            0.080963  
2            0.072497  
3            0.059129  
4            0.055845  
5            0.049813  
6            0.041665  
7            0.040149  
8            0.035664  
9          

In [6]:
feature_name_map = {
    "Drain Current (nA)__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.6_grad": r"$s$ Binned Quantile Change",
    "0_FFT mean coefficient_27": "FFT Mean Coefficient 27",
    "0_Absolute energy_grad": r"Absolute Energy in $s'$",
    "Drain Current (nA)__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""var""_grad" : r"$I_{DS}$ Linear Trend",
    'Drain Current (nA)__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.6_grad': r"$s'$ Binned Variance Quantile Change",
    'Drain Current (nA)__linear_trend__attr_"stderr"' : r"$s$ Linear Trend Std. Error",
    'Drain Current (nA)__energy_ratio_by_chunks__num_segments_10__segment_focus_1' : r"$s$ Energy Ratio",
    'Drain Current (nA)__agg_autocorrelation__f_agg_"median"__maxlag_40_grad': r"$s'$ Autocorrelation",
}

In [9]:
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from egfet.load_data import get_metadata_labels_groups, get_available_indices

sns.set(rc={'text.usetex' : True})

# Select the top 3 features by name in feature importance df
top_3_features = feature_importances_df.head(3)['feature'].tolist()  # Assuming the 'feature' column contains the feature names
print(top_3_features)
top_features = data[top_3_features]

print(top_3_features)
print(top_features.head())
# Rename the axis labels

custom_labels = [feature_name_map.get(x, x) for x in top_features.columns]
top_features.columns = custom_labels


# get labels
if 'labels' not in locals():
    indices, exclude, labels, groups = get_metadata_labels_groups(
            [
                "Experiment",
                "Concentration",
                "Trial",
            ],
            [],
            top_features.index.unique(),
            verbose=1,
            label_map_override = {
                "20221214_HP_HBS_CTH": 0,  # 0 corresponds to CTH
                "20230518_CTH_2kPEG": 0,
                "20240108_CTH_Hapt_2kPEG_24hr": 0,
                "20230929_1_BSA_Hapt_2kPEG": 2,  # 2 corresponds to BSA
                "20230929_2_BSA_Hapt_2kPEG": 2,
            },
        )

top_features["labels"] = labels
# change labels to strings 0 is Other, 1 is H4
top_features["labels"] = top_features["labels"].astype(str)
# change labels to strings 0 is Other, 1 is H4
top_features["labels"] = top_features["labels"].replace("0", "CTH")
top_features["labels"] = top_features["labels"].replace("1", "H4")
top_features["labels"] = top_features["labels"].replace("2", "BSA")
# Create the pairplot
plt.figure(figsize=(15, 10))
sns.pairplot(data=top_features, hue="labels", kind="kde")

# Show the plot
plt.show()


['Drain Current (nA)__energy_ratio_by_chunks__num_segments_10__segment_focus_0', 'Drain Current (nA)__agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"var"_grad', 'Drain Current (nA)__agg_autocorrelation__f_agg_"median"__maxlag_40']
['Drain Current (nA)__energy_ratio_by_chunks__num_segments_10__segment_focus_0', 'Drain Current (nA)__agg_linear_trend__attr_"slope"__chunk_len_50__f_agg_"var"_grad', 'Drain Current (nA)__agg_autocorrelation__f_agg_"median"__maxlag_40']
                                              Drain Current (nA)__energy_ratio_by_chunks__num_segments_10__segment_focus_0  \
Experiment Concentration Trial     Cycle                                                                                     
 0.747945   0.838852      0.721100  0.725885                                           0.662077                              
 0.029798   1.188880      0.013855  0.018363                                          11.983254                              
-0.269193  -0.243369  

ValueError: Length of values (1021) does not match length of index (7042)